In [2]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_energy as me
import model_electricity as ml
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import sqlalchemy
import sqlite3
from typing import Union

importlib.reload(ma)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(ml)


/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:898: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:898: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<module 'model_electricity' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_electricity.py'>

In [89]:
# read in the base time slice attribute
attr_time_slice = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/Energy/base_attribute_time_slice.csv")
attr_time_slice_cur = sa.model_attributes.get_other_attribute_table("time_slice")
# get the baseline specified demand profiles
fp_spd_excel = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/Energy/estimate_of_nemomod_specified_demand_profile.xlsx"




In [90]:


##  get the hour from the name
def get_hour(str_in: str) -> int:
    if "wd" in str_in:
        s = str_in.split("wd")[1]
    elif "we" in str_in:
        s = str_in.split("we")[1]
    else:
        s = - 1
    
    return int(s)

##  auto-generate a description
def format_desc(str_in: str, delta_hr: int = 6) -> str:
    hr = get_hour(str_in)
    if "wd" in str_in:
        season = str_in.split("wd")[0]
        day = "weekday"
    elif "we" in str_in:
        season = str_in.split("we")[0]
        day = "weekend"
    
    hr0 = str((hr + 22)%24).rjust(2, "0")
    hr1 = str((hr + 22 + delta_hr)%24).rjust(2, "0")
    
    str_out = f"{season} {day}s from {hr0}:00 to {hr1}:00"
    
    return str_out



dict_rename_l = {}
dict_hr_map = {}
##  build a map for new hour sets
for hr in range(24):
    hr_shift = hr + 2
    if hr_shift%24 < 6:
        dict_hr_map.update({hr: 0})
    elif hr_shift%24 < 12:
        dict_hr_map.update({hr: 6})
    elif hr_shift%24 < 18:
        dict_hr_map.update({hr: 12})
    else:
        dict_hr_map.update({hr: 18})
    
# specify the l order 
dict_l_order_by_hr = {0: 1, 6: 2, 12: 3, 18: 4}

##  build a map for time slice names
for time_slice in list(set(attr_time_slice["time_slice"])):
    
    hr = get_hour(time_slice)
    time_slice_new = time_slice.replace(str(hr), str(dict_hr_map[hr]))
    dict_rename_l.update({time_slice: time_slice_new})

    
##  perform aggregations on time slice attribute
field_ts = "time_slice"
field_ts_new = "time_slice_new"
attr_time_slice[field_ts_new] = attr_time_slice[field_ts].replace(dict_rename_l)
attr_time_slice["idn"] = range(len(attr_time_slice))

fields_grp = [field_ts_new, "tg1", "tg2"]
fields_sum = ["weight"]
dict_agg = dict(zip(fields_grp + ["idn"], ["first" for x in fields_grp + ["idn"]]))
dict_agg.update(dict(zip(fields_sum, ["sum" for x in fields_sum])))
attr_time_slice_new = attr_time_slice[list(dict_agg.keys())].groupby(
    fields_grp
).agg(
    dict_agg
).reset_index(
    drop = True
).sort_values(
    by = ["idn"]
).reset_index(
    drop = True
)

attr_time_slice_new["hr"] = attr_time_slice_new["time_slice_new"].apply(get_hour)
attr_time_slice_new["lorder"] = attr_time_slice_new["hr"].replace(dict_l_order_by_hr)
attr_time_slice_new.drop(["idn", "hr"], axis = 1, inplace = True)
attr_time_slice_new = attr_time_slice_new[[x for x in attr_time_slice.columns if x in attr_time_slice_new.columns]]
attr_time_slice_new["description"] = attr_time_slice_new[field_ts_new].apply(format_desc)
attr_time_slice_new.rename(columns = {"time_slice_new": "time_slice"}, inplace = True)

# order output
attr_time_slice_new = attr_time_slice_new[
    attr_time_slice_cur.table.columns
].rename(columns = {"time_slice": "``$TIME-SLICE$``"})

attr_time_slice_new.to_csv(attr_time_slice_cur.fp_table, index = None, encoding = "UTF-8")

In [102]:


##  SpecifiedDemandProfile

df_spd = pd.read_excel(fp_spd_excel, sheet_name = "SpecifiedDemandProfile")

##  aggregate to new specification
df_spd[field_ts_new] = df_spd["l"].replace(dict_rename_l)
fields_grp = [field_ts_new]
fields_sum = ["val"]
dict_agg = dict(zip(fields_grp + ["id"], ["first" for x in fields_grp + ["id"]]))
dict_agg.update(dict(zip(fields_sum, ["sum" for x in fields_sum])))
df_spd = df_spd[list(dict_agg.keys())].groupby(
    fields_grp
).agg(
    dict_agg
).reset_index(
    drop = True
).sort_values(
    by = ["id"]
).reset_index(
    drop = True
).rename(
    columns = {field_ts_new: "l"}
)


# get regions, fuels, and time periods, then initialize
attr_region = sa.model_attributes.get_other_attribute_table(sa.model_attributes.dim_region)
regions = attr_region.key_values;
df_spd_out = pd.DataFrame({"r": regions})
# add dummy key to both and merge
field_key_dummy = "key_merge"
df_spd[field_key_dummy] = 0
df_spd_out[field_key_dummy] = 0
df_spd_out = pd.merge(df_spd_out, df_spd, on = [field_key_dummy], how = "outer").drop([field_key_dummy], axis = 1).sort_values(by = ["r", "id"])
df_spd_out = df_spd_out.drop(["id"], axis = 1).reset_index(drop = True)


# write to output
fp_out = sa.dict_fp_csv_nemomod.get(sa.model_attributes.table_nemomod_specified_demand_profile)
df_spd_out.to_csv(fp_out, index = None, encoding = "UTF-8")

In [1168]:
# check TimeSlice attribute
attr_ts = sa.model_attributes.get_other_attribute_table("time_slice")
df_tmp = attr_ts.table.copy()


dict_repl = {"weekday": 5/7, "weekend": 2/7}
df_tmp["weight"] = df_tmp["tg2"].replace(dict_repl)

dict_denom = {}
for k in dict_repl.keys():
    df_filt = df_tmp[df_tmp["tg2"].isin([k])]
    dict_denom.update({k: len(df_filt)})

df_tmp["denom"] = df_tmp["tg2"].replace(dict_denom)
df_tmp["weight"] = np.array(df_tmp["weight"])/np.array(df_tmp["denom"])
df_tmp.drop(["denom"], axis = 1, inplace = True)
df_tmp.rename(columns = {"time_slice": "``$TIME-SLICE$``"}, inplace = True)
# uncomment to export
#df_tmp.to_csv(attr_ts.fp_table, index = None, encoding = "UTF-8")